In [1]:
import torch

In [2]:
a = torch.ones(3)
a

tensor([1., 1., 1.])

In [3]:
a[1]

tensor(1.)

In [4]:
a[2] = 2.
a

tensor([1., 1., 2.])

In [5]:
points = torch.tensor([4., 1., 5., 3., 2., 1.])
points

tensor([4., 1., 5., 3., 2., 1.])

In [6]:
float(points[0]), float(points[1])

(4.0, 1.0)

In [7]:
points = torch.zeros(3, 2)
points

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

In [8]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1.]])
points

tensor([[4., 1.],
        [5., 3.],
        [2., 1.]])

In [9]:
points.shape

torch.Size([3, 2])

In [10]:
points.size()

torch.Size([3, 2])

In [11]:
points[2, 1]

tensor(1.)

In [12]:
points[1]

tensor([5., 3.])

In [13]:
points[:, 1]

tensor([1., 3., 1.])

In [14]:
points[None] # note added dim

tensor([[[4., 1.],
         [5., 3.],
         [2., 1.]]])

### Named tensors

In [15]:
img_t = torch.randn(3, 5, 5) # [channels, rows, cols]
weights = torch.tensor([0.2126, 0.7152, 0.0722])
batch_t = torch.randn(2, 3, 5, 5) # [batch, chan, rows, cols]

In [16]:
img_grey_naive = img_t.mean(-3)
batch_grey_naive = batch_t.mean(-3)
img_grey_naive.shape, batch_grey_naive.shape

(torch.Size([5, 5]), torch.Size([2, 5, 5]))

In [17]:
weights.unsqueeze(-1)

tensor([[0.2126],
        [0.7152],
        [0.0722]])

In [18]:
weights.unsqueeze_(-1)

tensor([[0.2126],
        [0.7152],
        [0.0722]])

In [19]:
weights.unsqueeze(-1).unsqueeze_(-1)

tensor([[[[0.2126]]],


        [[[0.7152]]],


        [[[0.0722]]]])

In [20]:
unsqueezed_weights = weights.unsqueeze(-1).unsqueeze_(-1)
img_weight = img_t * unsqueezed_weights
batch_weights = batch_t * unsqueezed_weights
img_grey_weighted = img_weights.sum(-3)
batch_grey_weights = batch_weights.sum(-3)
[x.shape for x in (batch_weights, batch_t, unsqueezed_weights)]

RuntimeError: The size of tensor a (2) must match the size of tensor b (3) at non-singleton dimension 0

In [21]:
weights_named = torch.tensor([0.2126, 0.7163, 0.0733], 
                             names=['channels'])
weights_named

/var/folders/41/jhvt_zmd3nz3x5jsqblr221r0000gn/T/ipykernel_35400/1409454456.py:1: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  ../c10/core/TensorImpl.h:1408.)
  weights_named = torch.tensor([0.2126, 0.7163, 0.0733],


tensor([0.2126, 0.7163, 0.0733], names=('channels',))

In [22]:
img_named = img_t.refine_names(..., 'channels', 'rows', 'columns')
batch_named = batch_t.refine_names(..., 'channels', 'rows', 'columns')
print('img named:', img_named.shape, img_named.names)
print('batch named:', batch_named.shape, batch_named.names)

img named: torch.Size([3, 5, 5]) ('channels', 'rows', 'columns')
batch named: torch.Size([2, 3, 5, 5]) (None, 'channels', 'rows', 'columns')


In [23]:
weights_aligned = weights_named.align_as(img_named)
weights_aligned.shape, weights_aligned.names

(torch.Size([3, 1, 1]), ('channels', 'rows', 'columns'))

In [24]:
grey_named = (img_named * weights_aligned).sum('channels')
grey_named.shape, grey_named.names

(torch.Size([5, 5]), ('rows', 'columns'))

In [25]:
grey_plain = grey_named.rename(None)
grey_plain.shape, grey_plain.names

(torch.Size([5, 5]), (None, None))

### dtypes

In [26]:
double_points = torch.ones(10, 2, dtype=torch.double)
short_points = torch.tensor([[1, 2,], [3, 4]], dtype=torch.short)

In [27]:
double_points = torch.zeros(10, 2).double()
short_points = torch.ones(10, 2).short()

In [28]:
double_points = torch.zeros(10, 2).to(torch.double)
short_points = torch.ones(10, 2).to(dtype=torch.short)

In [29]:
points64 = torch.rand(4, dtype=torch.double)
points_short = points64.to(torch.short)
points_short

tensor([0, 0, 0, 0], dtype=torch.int16)

In [30]:
points64 * points_short

tensor([0., 0., 0., 0.], dtype=torch.float64)

### Tensor API

In [31]:
a = torch.ones(3, 2)
a_t = torch.transpose(a, 0, 1)
a.shape, a_t.shape

(torch.Size([3, 2]), torch.Size([2, 3]))

In [32]:
a_t = a.transpose(0, 1)
a_t.shape

torch.Size([2, 3])

## Storage
### Indexing into Storage

In [33]:
points = torch.tensor([[4., 1.], [5., 3.], [2., 1]])
points.storage()

 4.0
 1.0
 5.0
 3.0
 2.0
 1.0
[torch.FloatStorage of size 6]

In [34]:
points_storage = points.storage()
points_storage[0]

4.0

In [35]:
points.storage()[1]

1.0

In [36]:
points_storage[0] = 2.2
points

tensor([[2.2000, 1.0000],
        [5.0000, 3.0000],
        [2.0000, 1.0000]])

### Modifying Stored Values: In-place Ops

In [37]:
a = torch.ones(3, 2)
a.zero_()
a

tensor([[0., 0.],
        [0., 0.],
        [0., 0.]])

## Tensor Metadata: Size, Offset, Stride
### Veiws of another Tensor's Storage

In [39]:
points

tensor([[2.2000, 1.0000],
        [5.0000, 3.0000],
        [2.0000, 1.0000]])

In [38]:
second_point = points[1]
print(second_point)
second_point.storage_offset()

tensor([5., 3.])


2

In [41]:
second_point.size()

torch.Size([2])

In [42]:
points.stride()

(2, 1)

In [43]:
second_point.stride()

(1,)